# Premier League Analysis

* The Premier League (legal name: The Football Association Premier League Limited) is the highest level of the men's English football.
* In this project our goal is to use both sql and python in order to make useful tables and graphs about premier league season 2018/2019
* Database source : https://sqlsoccer.com/set-up-scripts/

### 1- Packages

In [4]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
import seaborn as sns
import geopandas as gpd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import json
from urllib.request import urlopen

In [18]:
pip install dash

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python\\Python39\\Lib\\site-packages\\~arkupsafe\\_speedups.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\python\python39\python.exe -m pip install --upgrade pip' command.


  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


### 2- connection to mysql

In [7]:
password = 'konodioda307'
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password= password,
  database = "PREMIERSHIP"
)

### 3- Teams Analysis

* Here using sql we will build the premier league Table

In [8]:
table = pd.read_sql_query("""select  rank() over(order by sum(points) desc) as 'Position', t.TEAM_NAME as club, count(TeamID) as MP, sum(wins) as W, sum(draws) as D, sum(loses) as L, sum(points) as Pts,
sum(goals) as GF, sum(goals_received) as GA, (sum(goals) - sum(goals_received)) as GD 
from
(SELECT g.GAME_ID, g.HOME_TEAM_ID as TeamID, g.HOME_SCORE as goals , g.AWAY_SCORE as goals_received,
case  when g.HOME_SCORE > g.AWAY_SCORE then 1 else 0 end as wins,
case when g.HOME_SCORE = g.AWAY_SCORE then 1 else 0 end as draws,
case when g.HOME_SCORE < g.AWAY_SCORE then 1 else 0 end as loses,
case 
 when g.HOME_SCORE > g.AWAY_SCORE then 3 
 when g.HOME_SCORE = g.AWAY_SCORE then 1
 else 0
 end as points
FROM game g
union all
select gg.GAME_ID ,gg.AWAY_TEAM_ID as TeamID, gg.AWAY_SCORE as goals , gg.HOME_SCORE as goals_received,
case  when gg.AWAY_SCORE > gg.HOME_SCORE then 1 end as wins,
case  when gg.AWAY_SCORE = gg.HOME_SCORE then 1 end as draws,
case  when gg.AWAY_SCORE < gg.HOME_SCORE then 1 end as loses,
case 
 when gg.AWAY_SCORE > gg.HOME_SCORE then 3 
 when gg.AWAY_SCORE = gg.HOME_SCORE then 1 
 else 0
 end as points
from game gg) as v 
join team as t
on t.TEAM_ID = TeamID
group by TeamID order by pts desc""",mydb,
            dtype= {'W':int, 'D':int,'L':int,'Pts': int,'GF':int, 'GA':int, 'GD':int})

<ipython-input-8-43c2805a7a4a>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table = pd.read_sql_query("""select  rank() over(order by sum(points) desc) as 'Position', t.TEAM_NAME as club, count(TeamID) as MP, sum(wins) as W, sum(draws) as D, sum(loses) as L, sum(points) as Pts,


* **MP**: Matches Played
* **W** : Wins
* **D** : Draws
* **L** : Loses
* **Pts** : Points
* **GF** : Goals For
* **GA** : Goals Against
* **GD** : Goals Difference

In [4]:
table

,Position,club,MP,W,D,L,Pts,GF,GA,GD
0,1,Manchester City,38,32,2,4,98,95,23,72
1,2,Liverpool,38,30,7,1,97,89,22,67
2,3,Chelsea,38,21,9,8,72,63,39,24
3,4,Tottenham Hotspur,38,23,2,13,71,67,39,28
4,5,Arsenal,38,21,7,10,70,73,51,22
5,6,Manchester United,38,19,9,10,66,65,54,11
6,7,Wolverhampton Wanderers,38,16,9,13,57,47,46,1
7,8,Everton,38,15,9,14,54,54,46,8
8,9,Leicester,38,15,7,16,52,51,48,3
9,9,West Ham United,38,15,7,16,52,52,55,-3


In [16]:
from dash import Dash, dcc, html
import plotly.express as px
from base64 import b64encode
import io

 # replace with your own data source
fig = px.bar(table,x = 'Pts',
             y = 'club',
             orientation='h', color= 'Pts',
             labels = {'Pts' : 'Points', 'W' : 'Wins'
                       ,'D':'Draws', 'L':'Loses'},
            title = 'Club Points Ranking', 
            hover_name = 'Position', hover_data = ['W','D','L'])
fig.update_layout(yaxis=dict(autorange="reversed"))
app = Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


#### Club points Ranking graph

In [17]:
fig = px.bar(table,x = 'Pts',
             y = 'club',
             orientation='h', color= 'Pts',
             labels = {'Pts' : 'Points', 'W' : 'Wins'
                       ,'D':'Draws', 'L':'Loses'},
            title = 'Club Points Ranking', 
            hover_name = 'Position', hover_data = ['W','D','L'])
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.write_html("Club points Ranking graph.html")

#### Clubs Goals Difference Ranking Graph

In [18]:
fig = px.bar(table,x = 'GD',
             y = 'club',
             orientation='h', color= 'GD',
             labels = {'GD': 'Goals Difference','GF' : 'Goals For','GA':'Goals Against'},
            title = 'Clubs Goals Differences', hover_data = ['GF','GA'])
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.write_html("Clubs Goals Differences.html")

#### Attack vs defence graph 

* Here we mesure the Attack by goals for while we mesure the Defense by Goals Against

In [19]:
fig = px.scatter(table, x = 'GA', y = 'GF',
                 hover_name = 'club',color = 'GD',
                 size = 'GF', title = 'Attack vs Defense' ,labels = {'GD': 'Goals Difference'
                                        ,'GF' : 'Goals For',
                                        'GA':'Goals Against'})
fig.write_html("Attack vs Defense.html")

#### Evolution of teams points 

* here we construct the initial table using sql 

In [21]:
teams_evo = pd.read_sql_query('''select game_date as date ,t.team_name,
tt.TEAM_NAME as 'Played against' ,Score ,Result,
sum(points) over(partition by team_name order by game_id) as cumulative_points 
from
(select *, g.HOME_TEAM_ID as team_id, g.AWAY_TEAM_ID as Played_against,
concat(g.Home_score,'-',g.AWAY_SCORE) as Score ,
case 
 when g.HOME_SCORE > g.AWAY_SCORE then 'Win' 
 when g.HOME_SCORE = g.AWAY_SCORE then 'Draw'
 else 'Lose'
 end as Result,
 case 
 when g.HOME_SCORE > g.AWAY_SCORE then 3
 when g.HOME_SCORE = g.AWAY_SCORE then 1
 else 0
 end as Points
 from game g 
union all 
select *, gg.AWAY_TEAM_ID as team_id, gg.HOME_TEAM_ID as Played_against ,
concat(gg.away_score,'-',gg.home_score) as Score,
 case 
 when gg.AWAY_SCORE > gg.HOME_SCORE then 'Win' 
 when gg.AWAY_SCORE = gg.HOME_SCORE then 'Draw'
 else 'Lose'
 end as Result,
case 
 when gg.AWAY_SCORE > gg.HOME_SCORE then 3 
 when gg.AWAY_SCORE = gg.HOME_SCORE then 1 
 else 0
 end as points
from game gg
) as x
join team t
on x.TEAM_ID = t.TEAM_ID
join team tt
on x.Played_against = tt.TEAM_ID''', mydb).set_index('team_name')
teams_evo#### Evolution of teams points graph

<ipython-input-21-44598638eb29>:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,date,Played against,Score,Result,cumulative_points
team_name,,,,,
Arsenal,2018-08-12,Manchester City,0-2,Lose,0.0
Arsenal,2018-08-18,Chelsea,2-3,Lose,0.0
Arsenal,2018-08-25,West Ham United,3-1,Win,3.0
Arsenal,2018-09-02,Cardiff,3-2,Win,6.0
Arsenal,2018-09-15,Newcastle,2-1,Win,9.0
...,...,...,...,...,...
Wolverhampton Wanderers,2019-04-20,Brighton And Hove Albion,0-0,Draw,48.0
Wolverhampton Wanderers,2019-04-24,Arsenal,3-1,Win,51.0
Wolverhampton Wanderers,2019-04-27,Watford,2-1,Win,54.0


#### Evolution of teams points graph

In [22]:
fig = go.Figure()
all = teams_evo.index.unique().tolist()
for club in all:
    fig.add_trace(go.Scatter(x = teams_evo.loc[club,].date,
                         y = teams_evo.loc[club,'cumulative_points'],name = club,
                         mode = 'lines+markers',
                       customdata = teams_evo.loc[club,['Played against',
                                                   'Score', 'Result']],
                            hovertemplate ="<br>".join([ 'Date : %{x}',
                                'Played Against : %{customdata[0]}',
                            'Score: %{customdata[1]}',
                            'Result : %{customdata[2]}'])))
fig.update_layout(
    title='clubs Points Evolution',
    xaxis= dict(title = 'Date' ),
    yaxis=dict(
        title='Points'))
    
fig.write_html("clubs Points Evolution.html")

### 4- Players Analysis

#### Players origins

In [23]:
players_nation = pd.read_sql_query('''SELECT n.ISO_A3 ,n.nation_name,
count(p.player_name) as number_of_players FROM player p
join nation n
on n.nation_id = p.nation_id
group by n.ISO_A3 order by n.ISO_A3 ''', mydb)
players_nation.head()

<ipython-input-23-5743039a960d>:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,ISO_A3,nation_name,number_of_players
0,ARG,Argentina,16
1,ARM,Armenia,1
2,AUS,Australia,2
3,AUT,Austria,4
4,BEL,Belgium,20


* Downloading a world map Geojson file 

In [24]:
with urlopen("https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson") as f:
    world_map = json.load(f)

In [12]:
world_map['features'][0]

{'type': 'Feature',
 'properties': {'ADMIN': 'Aruba', 'ISO_A3': 'ABW', 'ISO_A2': 'AW'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-69.99693762899992, 12.577582098000036],
     [-69.93639075399994, 12.53172435100005],
     [-69.92467200399994, 12.519232489000046],
     [-69.91576087099992, 12.497015692000076],
     [-69.88019771999984, 12.453558661000045],
     [-69.87682044199994, 12.427394924000097],
     [-69.88809160099993, 12.417669989000046],
     [-69.90880286399994, 12.417792059000107],
     [-69.93053137899989, 12.425970770000035],
     [-69.94513912699992, 12.44037506700009],
     [-69.92467200399994, 12.44037506700009],
     [-69.92467200399994, 12.447211005000014],
     [-69.95856686099992, 12.463202216000099],
     [-70.02765865799992, 12.522935289000088],
     [-70.04808508999989, 12.53115469000008],
     [-70.05809485599988, 12.537176825000088],
     [-70.06240800699987, 12.546820380000057],
     [-70.06037350199995, 12.556952216000113],
     [-70.0510961

* Making the plot

#### Players origins map

In [27]:
fig = px.choropleth(players_nation, geojson=world_map, 
                    locations='ISO_A3',featureidkey = 'properties.ISO_A3',
                    color='number_of_players',range_color = (0,50) ,
                    color_continuous_scale="Plasma",hover_name="nation_name",title = 'Premier League Players Origins'
                          )
fig.write_html("Players origins map.html")

#### Top Scorers table

In [28]:
top_scorers = pd.read_sql_query('''SELECT rank() over(order by sum(GOAL_ORDER) desc) as 'Position',n.NATION_NAME as NAT, t.TEAM_NAME as CLUB, p.player_name as NAME ,count(GOAL_ORDER) as GOALS from goal g
join player p
on p.player_id = g.player_id
join nation n 
on n.NATION_ID = p.NATION_ID
join team t 
on t.TEAM_ID = p.TEAM_ID
group by p.player_id order by GOALS desc limit 10''', mydb )
top_scorers

<ipython-input-28-5ae1fb02c242>:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Position,NAT,CLUB,NAME,GOALS
0,1,Gabon,Arsenal,Pierre-Emerick Aubameyang,22
1,2,Egypt,Liverpool,Mohamed Salah,22
2,3,Senegal,Liverpool,Sadio Mane,22
3,4,Argentina,Manchester City,Sergio Aguero,21
4,6,England,Leicester,Jamie Vardy,18
5,5,England,Manchester City,Raheem Sterling,17
6,7,England,Tottenham Hotspur,Harry Kane,17
7,13,Belgium,Chelsea,Eden Hazard,16
8,9,England,Bournemouth,Callum Wilson,14
9,8,England,Burnley,Ashley Barnes,13


#### Top scorers graph

In [29]:
fig = px.bar(top_scorers,x = 'GOALS',
             y = 'NAME',
             orientation='h', color= 'GOALS',
             labels = {'NAT' : 'Nationality'},
            title = 'Top Scorers', 
            hover_name = 'Position', hover_data = ['NAT','CLUB'])
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.write_html("Top Scorers.html")

#### Players goals Evolution

In [30]:
players_goals=pd.read_sql_query('''SELECT p.PLAYER_NAME , n.nation_name  ,
t.team_name  ,concat(tt2.team_name,' vs ',tt.team_name) as Game ,
concat(gg.home_score, '-',gg.away_score) as Score , 
 g.goal_order ,gg.GAME_DATE ,
 count(g.GOAL_ORDER) over(partition by g.PLAYER_ID order by gg.GAME_DATE) as goals FROM goal g
join game gg
on g.GAME_ID = gg.GAME_ID
join player p
on p.PLAYER_ID = g.PLAYER_ID
join team t
on t.team_id = p.team_id
join team tt
on tt.team_id = gg.away_team_id
join team tt2
on tt2.team_id = gg.home_team_id
join nation n
on n.nation_id = p.nation_id''', mydb).set_index('PLAYER_NAME')
players_goals

<ipython-input-30-c4db2ec542ce>:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,nation_name,team_name,Game,Score,goal_order,GAME_DATE,goals
PLAYER_NAME,,,,,,,
Victor Lindelof,Sweden,Manchester United,West Ham United vs Manchester United,3-1,2.0,2018-09-29,1
Victor Lindelof,Sweden,Manchester United,Manchester United vs Burnley,2-2,4.0,2019-01-29,2
Luke Shaw,England,Manchester United,Manchester United vs Leicester,2-1,2.0,2018-08-10,1
Juan Mata,Spain,Manchester United,Manchester United vs Newcastle,3-2,3.0,2018-10-06,1
Juan Mata,Spain,Manchester United,Manchester United vs Fulham,4-1,2.0,2018-12-08,2
...,...,...,...,...,...,...,...
Michy Batshuayi,Belgium,Crystal Palace,Crystal Palace vs Bournemouth,5-3,2.0,2019-05-12,5
Youri Tielemans,Belgium,Leicester,Leicester vs Fulham,3-1,1.0,2019-03-09,1
Youri Tielemans,Belgium,Leicester,Huddersfield vs Leicester,1-4,1.0,2019-04-06,2


#### Top 10 players goals evolution graph

In [31]:
fig = go.Figure()
top_scorers_names = top_scorers['NAME'].tolist()
for player in top_scorers_names:
        fig.add_trace(go.Scatter(x = players_goals.loc[player,].GAME_DATE,
                         y = players_goals.loc[player,'goals'],name = player,
                         mode = 'lines+markers',
                       customdata = players_goals.loc[player,['nation_name',
                                                   'team_name','Game','Score']],
                            hovertemplate ="<br>".join([ 'Date : %{x}',
                                'Nationality : %{customdata[0]}',
                            'Club: %{customdata[1]}',
                            'Match : %{customdata[2]}',
                            'Score : %{customdata[3]}',
                            'Total Goals until now : %{y}'])))
        fig.update_layout(
           title='Top 10 Scorers Total Goals Evolution',
           xaxis= dict(title = 'Date' ),
           yaxis=dict(
           title='Goals'))
    
fig.write_html("Top 10 Scorers Total Goals Evolution.html")